# Linear Regression with PyTorch

In [1]:
import numpy as np
import torch

# Prepare data

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In real project, input row data (CSV) as numpy array and then covert to tensor

In [4]:
# convert to tensors
inputs_tensor = torch.from_numpy(inputs)
targets_tensor = torch.from_numpy(targets)
inputs_tensor,targets_tensor

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

Linear Regession Model

In [5]:
# init weights and biases
'''
# torch.randn creates a tensor with the given shape,
# with elements picked randomly from a normal distribution
# with mean 0 and standard deviation 1.
'''
w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)
w, b

(tensor([[ 1.1785, -1.1212, -1.1480],
         [-0.7303,  0.4337,  0.2456]], requires_grad=True),
 tensor([ 0.0683, -1.1435], requires_grad=True))

In [6]:
# define linear model
'''
# @ represents matrix multiplication in PyTorch
'''
def model(x):
  return x @ w.t() + b

In [7]:
# show predict
preds = model(inputs_tensor)
preds,targets_tensor

(tensor([[ -38.3914,  -14.8369],
         [ -64.8336,  -13.7170],
         [-114.2358,    7.6819],
         [  29.5822,  -47.8989],
         [-106.6181,    7.2933]], grad_fn=<AddBackward0>),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

# Loss function

In [40]:
# MSE Loss
'''
# .numel returns the number of elements in a tensor
'''
def mse(pred, truth):
  diff = pred - truth
  return torch.sum(diff * diff) / diff.numel()

In [9]:
# Compute Loss
loss = mse(preds,targets_tensor)
loss

tensor(18432.1445, grad_fn=<DivBackward0>)

# Compute gradients

In [10]:
# Compute gradients
loss.backward()

In [11]:
# gradients for weights
w, w.grad

(tensor([[ 1.1785, -1.1212, -1.1480],
         [-0.7303,  0.4337,  0.2456]], requires_grad=True),
 tensor([[-10828.6416, -14041.6953,  -8262.5156],
         [ -8780.4912,  -9389.2568,  -5843.8091]]))

In [12]:
'''
# !!!IMPORTANT!!!
# before update next grad. we need to calling .zero()
# cause PyTorch accuumlates gradients
'''
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


# Adjust weights and biases using GD

In [13]:
# 1.Generate predictions
# 2.Calculate the loss
# 3.Compute gradients w.r.t the weights and biases
# 4.Adjust the weights by subtracting a small quantity proportional to the gradient
# 5.Reset the gradients to zero

# 1
preds = model(inputs_tensor)
# 2
loss = mse(preds,targets_tensor)
# 3
loss.backward()
# 4 & 5
with torch.no_grad():
  w -= w.grad * 1e-4
  b -= b.grad * 1e-4
  w.grad.zero_()
  b.grad.zero_()


torch.no_grad(*args, **kwargs)
Context-manager that disabled gradient calculation.

Disabling gradient calculation is useful for inference, when you are sure that you will not call :meth:Tensor.backward(). It will reduce memory consumption for computations that would otherwise have requires_grad=True.

In this mode, the result of every computation will have requires_grad=False, even when the inputs have requires_grad=True.

# Training

In [14]:
epochs = 100
lr = 1e-5

In [15]:
for e in range(epochs):
  preds = model(inputs_tensor)
  loss = mse(preds,targets_tensor)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * lr
    b -= b.grad * lr
    w.grad.zero_()
    b.grad.zero_()
  # print(e,loss)

In [17]:
preds,targets_tensor

(tensor([[ 68.4308,  70.2388],
         [ 82.1154,  98.2811],
         [100.7903, 138.5014],
         [ 85.7149,  37.4770],
         [ 64.0574, 114.4878]], grad_fn=<AddBackward0>), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

# LR uding PyTorch built-ins

In [2]:
import torch.nn as nn

In [3]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [4]:
# Dataset and DataLoader
from torch.utils.data import TensorDataset

In [6]:
# Define data set
'''
# TensorDataset combines data 
'''
train_ds = TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [7]:
from torch.utils.data import DataLoader

In [8]:
# define data Loader
batch_size = 5
'''
# DataLoader Data loader. Combines a dataset and a sampler, 
# and provides an iterable over the given dataset
'''
train_dl = DataLoader(train_ds,batch_size, shuffle=True)

In [10]:
# iteration example
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[73., 67., 43.],
        [91., 88., 64.],
        [91., 88., 64.],
        [73., 67., 43.],
        [69., 96., 70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [ 81., 101.],
        [ 56.,  70.],
        [103., 119.]])


# nn.Linear in-built LR

In [12]:
# Define model
'''
nn.Linear init w & b automatically
'''
model = nn.Linear(3,2)
model.weight, model.bias

(Parameter containing:
 tensor([[-0.1768,  0.4161, -0.5550],
         [ 0.0311,  0.3134, -0.1490]], requires_grad=True),
 Parameter containing:
 tensor([-0.4631,  0.1579], requires_grad=True))

In [14]:
# Parameters
'''
 .parameters is helpful method to return list all weight and bias in model
'''
list(model.parameters())

[Parameter containing:
 tensor([[-0.1768,  0.4161, -0.5550],
         [ 0.0311,  0.3134, -0.1490]], requires_grad=True),
 Parameter containing:
 tensor([-0.4631,  0.1579], requires_grad=True)]

In [16]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ -9.3546,  17.0163],
        [-15.4534,  21.0279],
        [  7.7226,  36.2150],
        [-21.1367,  11.2888],
        [-11.5658,  21.9581],
        [ -9.3546,  17.0163],
        [-15.4534,  21.0279],
        [  7.7226,  36.2150],
        [-21.1367,  11.2888],
        [-11.5658,  21.9581],
        [ -9.3546,  17.0163],
        [-15.4534,  21.0279],
        [  7.7226,  36.2150],
        [-21.1367,  11.2888],
        [-11.5658,  21.9581]], grad_fn=<AddmmBackward>)

# Loss function

In [17]:
# Import nn.functional that contains many useful loss functions
import torch.nn.functional as F

In [18]:
# Define loss function
# Direct import instead of implement by hand
loss_fn = F.mse_loss

In [21]:
loss = loss_fn(model(inputs),targets)
print(loss)

tensor(6959.1597, grad_fn=<MseLossBackward>)


# Optimizer

In [23]:
# Deine optimizer
opt = torch.optim.SGD(model.parameters(),lr = 1e-4)

# Train the model

In [29]:
def fit(epochs, model, loss_fn, opt, train_dl):
    for epoch in range(epochs):
        for x, y in train_dl:
            pred = model(x)
            loss = loss_fn(pred,y)
            loss.backward()
            opt.step()
            opt.zero_grad()
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, loss.item()))    

In [30]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 13.1481
Epoch [20/100], Loss: 36.2457
Epoch [30/100], Loss: 10.4093
Epoch [40/100], Loss: 52.1378
Epoch [50/100], Loss: 3.8174
Epoch [60/100], Loss: 1.5439
Epoch [70/100], Loss: 5.0449
Epoch [80/100], Loss: 4.4012
Epoch [90/100], Loss: 0.9627
Epoch [100/100], Loss: 1.6388


In [32]:
preds = model(inputs)
preds,targets

(tensor([[ 56.0663,  70.4670],
         [ 80.5399, 100.5478],
         [117.3745, 133.6111],
         [ 20.5035,  37.2555],
         [ 99.7652, 118.7711],
         [ 56.0663,  70.4670],
         [ 80.5399, 100.5478],
         [117.3745, 133.6111],
         [ 20.5035,  37.2555],
         [ 99.7652, 118.7711],
         [ 56.0663,  70.4670],
         [ 80.5399, 100.5478],
         [117.3745, 133.6111],
         [ 20.5035,  37.2555],
         [ 99.7652, 118.7711]], grad_fn=<AddmmBackward>), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.],
         [ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.],
         [ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))